## Applied Data Science Capstone

### Segmenting and Clustering Neighborhoods in Toronto

In [1]:
# Transformation ot the data from Wiki page into pandas
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#datasource
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
toronto_data = BeautifulSoup(source, 'lxml')

In [3]:
#create a new dataframe with 3 columns
column_titles = ['Postalcode','Borough','Neighborhood']
canada = pd.DataFrame(columns = column_titles)

In [4]:
canada.head()

,Postalcode,Borough,Neighborhood


In [5]:
#find the data
content = toronto_data.find('div', class_='mw-parser-output')
table = content.table.tbody

file = open("data.csv","w")

for tr in table.find_all('tr'):
    for td in tr.find_all('td'):
        text = td.getText().strip()+','
        file.write(text)
    for th in tr.findAll('th'):
        text = th.getText().strip()+','
        file.write(text)
    file.write('\n')
file.close()

In [6]:
canada = pd.read_csv('data.csv')
canada.head()

,Postcode,Borough,Neighbourhood,Unnamed: 3
0,M1A,Not assigned,Not assigned,NaN
1,M2A,Not assigned,Not assigned,NaN
2,M3A,North York,Parkwoods,NaN
3,M4A,North York,Victoria Village,NaN
4,M5A,Downtown Toronto,Harbourfront,NaN


In [7]:
#Ignore cells with a borough that is Not assigned
canada = canada.drop(canada[(canada.Borough == "Not assigned")].index)
canada.head()

,Postcode,Borough,Neighbourhood,Unnamed: 3
2,M3A,North York,Parkwoods,NaN
3,M4A,North York,Victoria Village,NaN
4,M5A,Downtown Toronto,Harbourfront,NaN
5,M5A,Downtown Toronto,Regent Park,NaN
6,M6A,North York,Lawrence Heights,NaN


In [8]:
#remove unnecessary column
canada.drop('Unnamed: 3', axis=1, inplace = True)
canada.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [9]:
#rename the columns according the example
canada.rename(columns={'Postcode': 'Postalcode', 'Neighbourhood': 'Neighborhood'}, inplace=True)
canada.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [10]:
i = 0
for i in range(0,canada.shape[0]):
    if canada.iloc[i][2] == 'Not assigned':
        canada.iloc[i][2] = canada.iloc[i][1]
        i = i+1

In [11]:
#combine neighborhood with the same postal code, join it with comma and reset index
canada = canada.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
canada.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
#If a cell has borough but a not assigned neighborhood, then the neighborhood will be the same as the borough
for i in range(len(canada)):
    rows_data=canada.iloc[i,:]
    if rows_data['Neighborhood'] == 'Not assigned':
        rows_data['Neighborhood'] = rows_data['Borough']

In [13]:
#results
canada.head(10)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [14]:
#shape
canada.shape

(103, 3)